In [357]:
import MySQLdb
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import time
import datetime
mysqlHost = "localhost"
mysqlDB="sys"
mongoHost="localhost"
user1="root"
password1='sampson'
import calendar
import sys
reload(sys)
sys.setdefaultencoding('utf-8')
from chainmap import ChainMap
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor

In [334]:
conn = MySQLdb.connect(host='localhost', user=user1, passwd=password1, db=mysqlDB, port=3306,charset = 'utf8')
cur = conn.cursor()
sql = 'SELECT * from weather where shi="杭州"'
weather = pd.read_sql(sql,conn,index_col='time')
aag = cur.execute('SELECT * from weather where shi="杭州"')
info = cur.fetchmany(aag)
cur.close()
conn.commit()
conn.close()

In [335]:
class_le = LabelEncoder()
weather["weather_day"] = class_le.fit_transform(weather["weather_day"])
weather["weather_night"] = class_le.fit_transform(weather["weather_night"])
weather["wind_day1"] = class_le.fit_transform(weather["wind_day1"])
weather["wind_day2"] = class_le.fit_transform(weather["wind_day2"])
weather["wind_night1"] = class_le.fit_transform(weather["wind_night1"])
weather["wind_night2"] = class_le.fit_transform(weather["wind_night2"])
weather.head()

,city_first,shi,weather_day,weather_night,tem_day,tem_night,wind_day1,wind_day2,wind_night1,wind_night2
time,,,,,,,,,,
2011-01-01,杭州,杭州,12,4,7,-2,3,0,5,8
2011-01-02,杭州,杭州,4,11,5,-1,3,9,5,8
2011-01-03,杭州,杭州,16,14,5,-1,3,9,5,8
2011-01-04,杭州,杭州,4,4,6,0,3,9,3,0
2011-01-05,杭州,杭州,14,14,5,0,3,0,3,2


In [336]:
weather[:datetime.datetime(2011, 1, 3).strftime("%Y-%m-%d")]

,city_first,shi,weather_day,weather_night,tem_day,tem_night,wind_day1,wind_day2,wind_night1,wind_night2
time,,,,,,,,,,
2011-01-01,杭州,杭州,12,4,7,-2,3,0,5,8
2011-01-02,杭州,杭州,4,11,5,-1,3,9,5,8
2011-01-03,杭州,杭州,16,14,5,-1,3,9,5,8


In [337]:
weather.loc[datetime.datetime(2011, 1, 3)]

city_first       鏉�窞
shi              鏉�窞
weather_day      16
weather_night    14
tem_day           5
tem_night        -1
wind_day1         3
wind_day2         9
wind_night1       5
wind_night2       8
Name: 2011-01-03 00:00:00, dtype: object

In [338]:
weather.index

DatetimeIndex(['2011-01-01', '2011-01-02', '2011-01-03', '2011-01-04',
               '2011-01-05', '2011-01-06', '2011-01-07', '2011-01-08',
               '2011-01-09', '2011-01-10',
               ...
               '2016-12-22', '2016-12-23', '2016-12-24', '2016-12-25',
               '2016-12-26', '2016-12-27', '2016-12-28', '2016-12-29',
               '2016-12-30', '2016-12-31'],
              dtype='datetime64[ns]', name=u'time', length=2192, freq=None)

In [339]:
t = 3
feathre = []
values = []
times = []
for idx in weather.index:  # 前后三天
    all_feature={}
    if datetime.datetime(2011, 1, 4) <= idx <= datetime.datetime(2016, 12, 27):
        weather_z = {}
        wind1_z = {}
        wind2_z = {}
        temp_z = {}
        d = 2*t+1
        day_night = {}
        temp_day_day = {}
        temp_night_night = {}
        temp_day_night = {}
        temp_night_day = {}
        time = {"time":idx}
        for i in range(1,d+1):
            forward = idx - datetime.timedelta(days=t) + datetime.timedelta(days=i-1)  # 七天时间

            weahter1 = {"weather_day" + str(i): weather.loc[forward]['weather_day'],"weather_night" + str(i): weather.loc[forward]['weather_night']}
            weather_z = dict(weather_z, **weahter1)

            wind1 = {"wind_day1" + str(i): weather.loc[forward]['wind_day1'],"wind_night1" + str(i): weather.loc[forward]['wind_night1']}
            wind1_z = dict(wind1_z, **wind1)

            wind2 = {"wind_day2" + str(i): weather.loc[forward]['wind_day2'],"wind_night2" + str(i): weather.loc[forward]['wind_night2']}
            wind2_z = dict(wind2_z, **wind2)

            temp = {"tem_day" + str(i): weather.loc[forward]['tem_day'],"tem_night" + str(i): weather.loc[forward]['tem_night']}
            temp_z = dict(temp_z, **temp)

            day_day1={"day" + str(i)+":day"+str(j+1): weather.loc[forward]['tem_day'] - weather.loc[forward + datetime.timedelta(days=j)]['tem_day'] for j in range(1,d+1-i) }  # 日温差
            temp_day_day=dict(temp_day_day,**day_day1)

            night_night1= {"night" + str(i)+":next_night"+str(i+j): weather.loc[forward]['tem_night'] -weather.loc[forward + datetime.timedelta(days=j)]['tem_night'] for j in range(1,d+1-i) }   # 夜温差
            temp_night_night = dict(temp_night_night,**night_night1)

            day_night1={"day" + str(i)+":night"+str(i+j): weather.loc[forward]['tem_day'] - weather.loc[forward+ datetime.timedelta(days=j)]['tem_night'] for j in range(0,d-i)}   # 日夜温差
            temp_day_night=dict(temp_day_night,**day_night1)

            night_day1={"night" + str(i)+":next_day"+str(i+j): weather.loc[forward]['tem_night'] -weather.loc[forward + datetime.timedelta(days=j)]['tem_day'] for j in range(1,d+1-i)}  # 大量日夜温差  1-7天
            temp_night_day=dict(temp_night_day,**night_day1)
        idx={}
        all_feature=dict(time.items()+weather_z.items()+wind1_z.items()+wind2_z.items()+temp_z.items()+temp_day_day.items()+temp_night_night.items()+temp_day_night.items()+temp_night_day.items())          #多个字段字典合并
        feathre = all_feature.keys()
        value = all_feature.values()
        values.append(value)

    else:
        continue

In [340]:
all_values = pd.DataFrame(values,columns=feathre)
all_values.index=all_values["time"]
end_data = pd.merge(weather,all_values,left_index=True,right_index=True,how='inner')
end_data.drop('time',axis=1,inplace=True)
end_data.head()

,city_first,shi,weather_day,weather_night,tem_day,tem_night,wind_day1,wind_day2,wind_night1,wind_night2,...,night1:next_night3,night1:next_night2,night1:next_night7,night1:next_night6,night1:next_night5,night1:next_night4,day3:day4,day3:day5,day3:day2,day3:day3
time,,,,,,,,,,,,,,,,,,,,,
2011-01-04,杭州,杭州,4,4,6,0,3,9,3,0,...,-1,-1,-1,0,-2,-2,0,-1,-1,0
2011-01-05,杭州,杭州,14,14,5,0,3,0,3,2,...,-1,0,-1,0,1,-1,0,1,1,1
2011-01-06,杭州,杭州,4,12,5,-2,3,0,3,0,...,-1,-1,0,-1,0,1,0,-1,0,-1
2011-01-07,杭州,杭州,12,4,6,-1,0,9,5,8,...,2,0,1,1,0,1,-1,1,-1,0
2011-01-08,杭州,杭州,4,14,5,0,3,9,0,8,...,1,2,2,1,1,0,2,1,1,0


In [341]:
hangzhou_date = weather[[5]]

In [342]:
infection_data = pd.read_csv("./infection.csv")
infection_data["start_date"]=pd.to_datetime(infection_data["start_date"])
infection_data.index=infection_data['start_date']
infection_data.drop("Unnamed: 0",axis=1,inplace=True)
infection_data.drop("start_date",axis=1,inplace=True)
infection_data.head()

,count
start_date,
2011-01-01,8
2011-01-02,8
2011-01-03,5
2011-01-04,7
2011-01-05,6


In [343]:
feathre = []
values = []
times = []
d = 11
for idx in infection_data.index:  # 前7天 后三天
    if datetime.datetime(2011, 1, 10) <= idx <= datetime.datetime(2016, 12, 20):
        
        disease_z = {}
        time = {"time": idx}
        for i in range(0, d):
            forward = idx - datetime.timedelta(days=(7 - i))  # 七天时间
            try:
                infection = {"disease_day" + str(i - 7): infection_data.loc[forward]['count']}
            except:
                infection = {"disease_day" + str(i - 7): 0}
            disease_z = dict(disease_z, **infection)
        all_feature=dict(time.items()+disease_z.items())          #多个字段字典合并
        feathre = all_feature.keys()
        value = all_feature.values()
        values.append(value)
    else:
        continue
infection_values = pd.DataFrame(values, columns=feathre)

In [344]:
infection_values.head()
infection_values.shape

(2148, 12)

In [345]:
infection_values.index = infection_values['time']
infection_end = pd.merge(end_data,infection_values,left_index=True,right_index=True,how='left')
infection_end.drop('time',axis=1,inplace=True)
infection_end.head(100)

,city_first,shi,weather_day,weather_night,tem_day,tem_night,wind_day1,wind_day2,wind_night1,wind_night2,...,disease_day2,disease_day1,disease_day0,disease_day-3,disease_day-2,disease_day-1,disease_day-7,disease_day-6,disease_day-5,disease_day-4
time,,,,,,,,,,,,,,,,,,,,,
2011-01-04,杭州,杭州,4,4,6,0,3,9,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-05,杭州,杭州,14,14,5,0,3,0,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-06,杭州,杭州,4,12,5,-2,3,0,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-07,杭州,杭州,12,4,6,-1,0,9,5,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-08,杭州,杭州,4,14,5,0,3,9,0,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-09,杭州,杭州,14,4,6,-1,3,9,5,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-10,杭州,杭州,4,14,4,-1,3,9,3,0,...,2.0,3.0,9.0,5.0,2.0,2.0,5.0,7.0,6.0,4.0
2011-01-11,杭州,杭州,4,4,5,-2,3,9,5,8,...,6.0,2.0,3.0,2.0,2.0,9.0,7.0,6.0,4.0,5.0
2011-01-12,杭州,杭州,12,12,6,-1,3,9,0,8,...,0.0,6.0,2.0,2.0,9.0,3.0,6.0,4.0,5.0,2.0


In [403]:
infection_end=infection_end.fillna(0)
names = infection_end.columns
names = list(names)
names.remove("disease_day0")
names.remove("disease_day1")
names.remove("disease_day2")
names.remove("disease_day3")
infection_end.head(30)

,city_first,shi,weather_day,weather_night,tem_day,tem_night,wind_day1,wind_day2,wind_night1,wind_night2,...,disease_day2,disease_day1,disease_day0,disease_day-3,disease_day-2,disease_day-1,disease_day-7,disease_day-6,disease_day-5,disease_day-4
time,,,,,,,,,,,,,,,,,,,,,
2011-01-04,杭州,杭州,4,4,6,0,3,9,3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-05,杭州,杭州,14,14,5,0,3,0,3,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-06,杭州,杭州,4,12,5,-2,3,0,3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-07,杭州,杭州,12,4,6,-1,0,9,5,8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-08,杭州,杭州,4,14,5,0,3,9,0,8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-09,杭州,杭州,14,4,6,-1,3,9,5,8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-10,杭州,杭州,4,14,4,-1,3,9,3,0,...,2.0,3.0,9.0,5.0,2.0,2.0,5.0,7.0,6.0,4.0
2011-01-11,杭州,杭州,4,4,5,-2,3,9,5,8,...,6.0,2.0,3.0,2.0,2.0,9.0,7.0,6.0,4.0,5.0
2011-01-12,杭州,杭州,12,12,6,-1,3,9,0,8,...,0.0,6.0,2.0,2.0,9.0,3.0,6.0,4.0,5.0,2.0


In [388]:
X_train,X_test,y_train,y_test = train_test_split(infection_end[names[2:]],infection_end["disease_day3"],test_size=0.3,random_state=42)

clf = DecisionTreeClassifier(max_depth=6,min_samples_split=6)
clf.fit(X_train,y_train)
clf.score(X_train,y_train)

0.23021582733812951

In [389]:
clf.score(X_test,y_test)

0.14939024390243902

In [390]:
forest = RandomForestClassifier(n_estimators=3000,random_state=42,n_jobs=3)
forest.fit(X_train,y_train)
forest.score(X_train,y_train)

1.0

In [391]:
clf.score(X_test,y_test)

0.14939024390243902

In [392]:
tr_predict = clf.predict(X_test)
print(classification_report(y_test,tr_predict))

In [393]:
forest = RandomForestRegressor(random_state=2)
forest.fit(X_train,y_train)
forest.score(X_train,y_train)

0.83099926844176186

In [394]:
forest.score(X_test,y_test)

0.088965301234012473

In [401]:
est = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01,max_depth=1, random_state=0, loss='ls')
est.fit(X_train,y_train)
est.score(X_train,y_train)

0.25648013045502116

In [402]:
est.score(X_test,y_test)

0.20019719824052662